LEER LOS ARCHIVOS

In [2]:
#IMPORTAR LAS LIBRERIAS
import json
import pandas as pd
import ast

In [20]:
#LEER EL PRIMER ARCHIVO DE JSON
rows=[]
with open("australian_user_reviews.json",encoding="MacRoman")as file:
    for line in file.readlines():
        rows.append(ast.literal_eval(line))
game_reviews=pd.DataFrame(rows)

In [5]:
#LEER EL SEGUNDO ARCHIVO
rows=[]
with open("australian_users_items.json",encoding="MacRoman")as file:
    for line in file.readlines():
        rows.append(ast.literal_eval(line))
game_items=pd.DataFrame(rows)

In [3]:
#LEER EL TERCER ARCHIVO
steam=pd.read_json("output_steam_games.json", lines=True)

In [ ]:
TRABAJAR EL PRIMER ARCHIVO

In [21]:
#USAR EXPLODE PARA SEPARAR LOS DICCIONARIOS EN VARIAS FILAS
prueba1=game_reviews.explode("reviews")

In [22]:
#VER EL CONTENIDO DE LA PRIMERA FILA DE LA COLUMNA REVIEWS
game_reviews.iloc[0,2]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [23]:
#CONVERIR LA COLUMNA DE DICCIONARIOS EN COLUMNAS DE UN DATAFRAME
pd1=prueba1["reviews"].apply(pd.Series)

In [24]:
# UNIR LAS COLUMNAS Y ELIMINAR LA COLUMNA CON LOS DICCIONARIOS
final=pd.concat([game_reviews,pd1], axis="columns")

In [25]:
#eliminar la columna reviews
reviews=final.drop(["reviews"], axis=1)

In [27]:
#HABIA UNA FILA QUE SE PUSO AL FINAL CON EL VALOR DE O, Y NO SE VA A INCLUIR EN EL DATASET
reviews=reviews.iloc[:,0:9]

In [30]:
reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [29]:
#PASAR EL DATAFRAME A UN ARCHIVO CSV
reviews.to_csv("game_reviews.csv", index=False)

TRABAJAR EL SEGUNDO ARCHIVO

In [4]:
#ELIMINAR LAS NAN DEL ARCHIVO
game_items.dropna(thresh=5, inplace=True)

In [ ]:
#VER QUE HAY EN LA PRIMERA FILA Y EN LA COLUMNA DE ITEMS
game_items["items"][0]

In [10]:
#SEPARAR EL CODIGO PARA SEPARAR LA COLUMNA ANIDADA
expanded_records = []

for index,row in game_items.iterrows():
    user_id  = row['user_id']
    
    steam_id = row['steam_id']
    user_url = row['user_url']
    items_count = row["items_count"]
    items_list = row['items']

    for it in items_list:
        # diccionario del nuevo df expandido
        new_item = {    'steam_id' : steam_id ,
                        'user_url' : user_url,
                        "items_count": items_count,
                        # **it
                        'item_id'  : it['item_id'],
                        'item_name'  : it['item_name'],
                        'playtime_forever'  : it['playtime_forever'],
                        'playtime_2weeks'  : it['playtime_2weeks']
                    }
        expanded_records.append(new_item)

game_items2 = pd.DataFrame(expanded_records)

In [12]:
#PASAR EL ARCHIVO A CSV
game_items2.to_csv("game_items.csv", index=False)

TRABAJAR EL TERCER ARCHIVO

In [4]:
#ELIMINAR LOS NAN
steam.dropna(how="all", inplace=True)

In [5]:
#PASAR LOS ARCHIVOS A CSV
steam.to_csv("steam_games.csv", index=False)

In [6]:
steam

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
